In [12]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
# manuplation hourly data to remove minutes
raw_data = pd.read_csv('raw_data.csv')
time_dt = pd.to_datetime(raw_data['time'], format='%d.%m.%Y %H:%M')
time_dt_hourbase = time_dt

for i in range(0, len(time_dt)):
    mmin = pd.Timedelta(minutes=time_dt[i].minute)
    time_dt_hourbase[i] = time_dt[i] - mmin
# Creating time domain for bus working hours
data_new = raw_data.set_index(time_dt_hourbase)
start_date = min(data_new.index)
end_date = max(data_new.index)

time_domain = pd.date_range(start=start_date, end=end_date, freq='H')
working_time_domain = time_domain[time_domain.hour > 6]
working_time_domain = working_time_domain[working_time_domain.hour < 17]
working_time_domain = pd.DataFrame(data=working_time_domain, columns=['rev_time'])
working_time_domain = working_time_domain.set_index(working_time_domain['rev_time'])
# seperating data set for each municiplaity ID and make domain same

data_new_sorted = data_new.sort_values(by=['user'], ascending=False)
data_cleared = pd.DataFrame()
for i in range(0, 10):
    temp_df = data_new_sorted[data_new_sorted['place_id'] == i]
    temp_ind = temp_df.index.duplicated(keep='first')
    last_df = temp_df[temp_ind == False]
    last_df = last_df.sort_index()
    last_df = pd.concat([working_time_domain, last_df], axis=1)
    last_df['place_id'] = last_df['place_id'].fillna(i)
    data_cleared = data_cleared.append(last_df, ignore_index=False)
# continiuty check
date_freq = pd.DataFrame()
for i in range(0, 10):
    y = data_cleared['user'][data_cleared['place_id'] == i]
    date_freq['place_' + str(i)] = pd.Series(working_time_domain.index.date[y.isna() == True]).value_counts()

# day should be assumed completely = 20/06, 21/06, 31/07,03/08,04/08
missing_dates = date_freq.index[date_freq['place_0'] == 10]
# other missing data only one hour on a day
filled_data = pd.DataFrame()
for i in range(0, 10):
    y = data_cleared[data_cleared['place_id'] == i]
    for k in range(0, len(y)):
        if y.index[k].date() in missing_dates:
            y.time[k] = y.rev_time[k]
            y.user[k] = (y.user[(k - 70)]+ y.user[(k -140)])*0.5
            y.capacity[k] = y.capacity[k - 1]
        elif pd.isna(y.user[k]) and (y.index[k].hour == 7 or y.index[k].hour == 16):
            y.time[k] = y.rev_time[k]
            y.user[k] = (y.user[k - 10]+y.user[k-20])*0.5
            y.capacity[k] = y.capacity[k - 1]
        elif pd.isna(y.user[k]):
            y.time[k] = y.rev_time[k]
            y.user[k] = (y.user[k - 1] + y.user[k + 1]) * 0.5
            y.capacity[k] = y.capacity[k - 1]
    filled_data = filled_data.append(y, ignore_index=False)



results = pd.DataFrame(columns=['Place_id','Forecasted WK','Fourier_Forecast','Fourier_Fit','MLR_Forecast','MLR_Fit'])
for wk in range (0,2):
    for i in range(0,10):
        N = 770
    #Define regressors
        t0 = np.ones((N, 1))
        t1 =np.transpose(np.linspace(1,N,N))
    # Harmonics of month
    # T = 1 ay = 28 days * 10 hours
        Tay = 28 * 10
        om = 2 * np.pi / Tay
        s1 = np.sin(1 * om * t1)
        c1 = np.cos(1 * om * t1)
        s2 = np.sin(2 * om * t1)
        c2 = np.cos(2 * om * t1)
        s3 = np.sin(3 * om * t1)
        c3 = np.cos(3 * om * t1)
    # Harmonics of 1 week % Tgun = 10 hours
        Thafta = 10 * 7
        omh = 2 * np.pi / Thafta
        sh1 = np.sin(1 * omh * t1)
        ch1 = np.cos(1 * omh * t1)
        sh2 = np.sin(2 * omh * t1)
        ch2 = np.cos(2 * omh * t1)
        sh3 = np.sin(3 * omh * t1)
        ch3 = np.cos(3 * omh * t1)
        sh4 = np.sin(4 * omh * t1)
        ch4 = np.cos(4 * omh * t1)
        sh5 = np.sin(5 * omh * t1)
        ch5 = np.cos(5 * omh * t1)
        sh6 = np.sin(6 * omh * t1)
        ch6 = np.cos(6 * omh * t1)
        sh8 = np.sin(8 * omh * t1)
        ch8 = np.cos(8 * omh * t1)
        sh9 = np.sin(9 * omh * t1)
        ch9 = np.cos(9 * omh * t1)
        sh10 = np.sin(10 * omh * t1)
        ch10 = np.cos(10 * omh * t1)
        sh11 = np.sin(11 * omh * t1)
        ch11 = np.cos(11 * omh * t1)
        sh12 = np.sin(12 * omh * t1)
        ch12 = np.cos(12 * omh * t1)
        sh13 = np.sin(13 * omh * t1)
        ch13 = np.cos(13 * omh * t1)
    # Harmonics of 1 gun
        Tgun = 10;
        omd = 2 * np.pi / Tgun
        sd1 = np.sin(omd * t1)
        cd1 = np.cos(omd * t1)
        sd2 = np.sin(2 * omd * t1)
        cd2 = np.cos(2 * omd * t1)
        sd3 = np.sin(3 * omd * t1)
        cd3 = np.cos(3 * omd * t1)
        sd4 = np.sin(4 * omd * t1)
        cd4 = np.cos(4 * omd * t1)
    #include modulation Fourier Coeff
        FFour=np.column_stack((s1,c1,s2,c2,s3,c3,sh1,ch1 ,sh2,ch2,
                          sh3, ch3, sh4, ch4, sh5, ch5, sh6, ch6, sh8, ch8,
                          sh9, ch9, sh10, ch10, sh11, ch11, sh12, ch12, sh13, ch13,
                          sd1, cd1, sd2, cd2, sd3, cd3, sd4, cd4,
                          np.multiply(s1,sd1),np.multiply(c1,sd1),np.multiply(s1,cd1),np.multiply(c1,cd1),
                          np.multiply(s1,sd2),np.multiply(c1,sd2),np.multiply(s1,cd2),np.multiply(c1,cd2),
                          np.multiply(s1,sd3),np.multiply(c1,sd3),np.multiply(s1,cd3),np.multiply(c1,cd3),
                          np.multiply(s1,sd4),np.multiply(c1,sd4),np.multiply(s1,cd4),np.multiply(c1,cd4),
                          np.multiply(s2, sd1), np.multiply(c2, sd1), np.multiply(s2, cd1), np.multiply(c2, cd1),
                          np.multiply(s2, sd2), np.multiply(c2, sd2), np.multiply(s2, cd2), np.multiply(c2, cd2),
                          np.multiply(s2, sd3), np.multiply(c2, sd3), np.multiply(s2, cd3), np.multiply(c2, cd3),
                          np.multiply(s2, sd4), np.multiply(c2, sd4), np.multiply(s2, cd4), np.multiply(c2, cd4),
                          np.multiply(s3, sd1), np.multiply(c3, sd1), np.multiply(s3, cd1), np.multiply(c3, cd1),
                          np.multiply(s3, sd2), np.multiply(c3, sd2), np.multiply(s3, cd2), np.multiply(c3, cd2),
                          np.multiply(s3, sd3), np.multiply(c3, sd3), np.multiply(s3, cd3), np.multiply(c3, cd3),
                          np.multiply(s3, sd4), np.multiply(c3, sd4), np.multiply(s3, cd4), np.multiply(c3, cd4)))

        Ftotal =np.column_stack((t0,t1,FFour))
        # Exclude last two weeks
        n = 770 - 70 * (wk+1);
        FM = Ftotal[0:n,:]
        S = filled_data.user[filled_data['place_id'] == i]
        S=S.to_numpy()
        Shorizon = S[0:n] #Base Actual Data
        Stest = S[n:n + 70]
        FT = Ftotal[n:n + 70,:] # Fourier for test horizon
        test_coeff=np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(FM),FM)),np.transpose(FM)),Shorizon)

        model_fit=np.matmul(FM , test_coeff)
        test=np.matmul(FT, test_coeff)
        fark_fline=Stest-test
        Mse=np.sqrt(np.matmul(np.transpose(fark_fline),fark_fline))
        Mpe=np.mean((np.absolute(np.divide(fark_fline,Stest))))*100
        fark_fline1=model_fit-Shorizon
        Mpe_fit = np.mean((np.absolute(np.divide(fark_fline1, Shorizon)))) * 100


        hours = np.tile(np.eye(10), (77, 1))
        a = np.concatenate((np.ones((10, 1)), np.zeros((10, 6))),axis=1)
        a = np.concatenate((a,np.roll(a, 1),np.roll(a, 2),np.roll(a, 3),np.roll(a, 4),np.roll(a, 5),np.roll(a, 6)),axis=0)
        week = np.tile(a, (11, 1));
        hours = hours[:,0:9];week = week[:, 0:6];
        Freg = np.column_stack((t0, t1, hours, week));

        FM = Freg[0:n,:]
        Shorizon=S[0:n]
        Stest = S[n:n + 70]
        FT = Freg[n:n + 70,:]
        test_coeff_reg=np.matmul(np.matmul(np.linalg.inv(np.matmul(np.transpose(FM),FM)),np.transpose(FM)),Shorizon)
        model_fit_reg=np.matmul(FM , test_coeff_reg)
        test_reg = np.matmul(FT, test_coeff_reg)

        fark_fline_reg = Stest - test_reg
        Mse_reg = np.sqrt(np.matmul(np.transpose(fark_fline_reg), fark_fline_reg))
        Mpe_reg = np.mean((np.absolute(np.divide(fark_fline_reg, Stest)))) * 100
        fark_fline2 = model_fit_reg - Shorizon
        Mpe_reg_fit = np.mean((np.absolute(np.divide(fark_fline2, Shorizon)))) * 100


        r=pd.DataFrame(data=[[i,76+wk,Mpe,Mpe_fit,Mpe_reg,Mpe_reg_fit]],columns=['Place_id','Forecasted WK','Fourier_Forecast','Fourier_Fit','MLR_Forecast','MLR_Fit'])
        results=results.append(r,ignore_index=True)

results.to_csv(r'results.csv', sep=',')
print(results)

   Place_id Forecasted WK  Fourier_Forecast  Fourier_Fit  MLR_Forecast  \
0         0            76         32.215841    18.087475     30.666552   
1         1            76          8.312250    11.504362      8.137491   
2         2            76         12.898409    11.735514     10.890132   
3         3            76         16.825211    15.873544     19.125181   
4         4            76         17.103791    18.432274     21.222802   
5         5            76         35.074038    23.889412     38.905481   
6         6            76         26.323191    12.157969     22.850619   
7         7            76          8.248844     8.454503      7.673241   
8         8            76         11.703849    10.257676     13.455173   
9         9            76          8.784046     8.723684      8.659284   
10        0            77         16.856421    18.535317     10.878098   
11        1            77          9.610129    11.793759      7.086301   
12        2            77         20.6

In [ ]:
#Explanation & comments : I developped two model both based on regression but one of them coefficients comes from Fourier Expansion 
#because if daily curves of the data are checked there is weekly periyodik cycle
#so Fourier expansion with modulation is powerful method to modelling this kind of data. 
#Other model, multiple linear regression. When we compare results for forecast and modelling, Fourier Model's errors
#are the lower for modelling existing data while forecast errors are quite high.
#I didn't make any residual analysis because this just to show how I approach to any problem. 
#Also, I didn't use any algrithm packages to develop more complex model.I shared my matlab source code as well in .txt file. 